HW2 分類問題


step.1 CRISP-DM 框架

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer


step.2 從網路取得資料（train.csv、test.csv）

In [2]:
##從github找到Titanic-Machine-Learning-from-Disaster train.csv test.csv Raw檔 載入

test_data=pd.read_csv("https://raw.githubusercontent.com/ashishpatel26/Titanic-Machine-Learning-from-Disaster/refs/heads/master/input/test.csv")
train_data=pd.read_csv("https://raw.githubusercontent.com/ashishpatel26/Titanic-Machine-Learning-from-Disaster/refs/heads/master/input/train.csv")

In [3]:
print(test_data.info(),"\n")
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   P

資料探勘與前處理

In [4]:
total_data=pd.DataFrame()

total_data = pd.concat([train_data, test_data], ignore_index=True)

total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [5]:
# 刪除不需要的欄位並處理缺失值
train_data = train_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)

# 編碼類別變量
label_enc = LabelEncoder()
train_data['Sex'] = label_enc.fit_transform(train_data['Sex'])
train_data['Embarked'] = label_enc.fit_transform(train_data['Embarked'])

# 分割特徵與目標變量
X = train_data.drop('Survived', axis=1)
y = train_data['Survived']

<ipython-input-5-43d32c30d04b>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
<ipython-input-5-43d32c30d04b>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

step.3 修改程式碼使用RFE、SelectKBest、optuna選擇最佳特徵

RFE 是一種特徵選擇的包裝方法，會遞迴地移除較不重要的特徵

In [6]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# 初始化模型和 RFE 選擇器
model = LogisticRegression(max_iter=200)
rfe_selector = RFE(model, n_features_to_select=5, step=1)
rfe_selector.fit(X, y)

# 獲取選擇的特徵
selected_features_rfe = X.columns[rfe_selector.support_]
print("RFE 選擇的特徵:", selected_features_rfe)



RFE 選擇的特徵: Index(['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked'], dtype='object')


SelectKBest 是一種過濾方法，會根據指定的統計測試選擇特徵

In [7]:
from sklearn.feature_selection import SelectKBest, chi2

# 使用 SelectKBest 選擇前 5 個特徵
kbest_selector = SelectKBest(score_func=chi2, k=5)
X_kbest = kbest_selector.fit_transform(X, y)
selected_features_kbest = X.columns[kbest_selector.get_support()]
print("SelectKBest 選擇的特徵:", selected_features_kbest)


SelectKBest 選擇的特徵: Index(['Pclass', 'Sex', 'Age', 'Fare', 'Embarked'], dtype='object')


Optuna 是一個優化庫，可以通過調整超參數來同時選擇最佳特徵。

In [8]:
!pip install optuna
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    selected_columns = []
    for i in X.columns:
        if trial.suggest_categorical(f"use_{i}", [True, False]):
            selected_columns.append(i)

    if len(selected_columns) == 0:
        return 0.0  # 避免空特徵集

    # 創建和評估模型
    model = RandomForestClassifier()
    X_selected = X[selected_columns]
    score = cross_val_score(model, X_selected, y, cv=5, scoring="accuracy").mean()
    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)
print("Optuna 選擇的最佳特徵:", study.best_params)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00


[I 2024-10-29 03:51:31,556] A new study created in memory with name: no-name-7dfc6c1a-d4e5-415a-9011-892b524c9984
[I 2024-10-29 03:51:34,390] Trial 0 finished with value: 0.6880610131190761 and parameters: {'use_Pclass': False, 'use_Sex': False, 'use_Age': False, 'use_SibSp': False, 'use_Parch': True, 'use_Fare': True, 'use_Embarked': False}. Best is trial 0 with value: 0.6880610131190761.
[I 2024-10-29 03:51:37,642] Trial 1 finished with value: 0.6779047140794676 and parameters: {'use_Pclass': False, 'use_Sex': False, 'use_Age': True, 'use_SibSp': True, 'use_Parch': False, 'use_Fare': True, 'use_Embarked': False}. Best is trial 0 with value: 0.6880610131190761.
[I 2024-10-29 03:51:41,354] Trial 2 finished with value: 0.8159939740129308 and parameters: {'use_Pclass': True, 'use_Sex': True, 'use_Age': True, 'use_SibSp': False, 'use_Parch': True, 'use_Fare': True, 'use_Embarked': False}. Best is trial 2 with value: 0.8159939740129308.
[I 2024-10-29 03:51:45,186] Trial 3 finished with val

Optuna 選擇的最佳特徵: {'use_Pclass': True, 'use_Sex': True, 'use_Age': True, 'use_SibSp': False, 'use_Parch': True, 'use_Fare': True, 'use_Embarked': False}


step.4 混淆矩陣。準確性

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

# 將數據分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X[selected_features_rfe], y, test_size=0.2, random_state=42)

# 訓練模型
model = RandomForestClassifier()
model.fit(X_train, y_train)

# 預測
y_pred = model.predict(X_test)

# 混淆矩陣與準確性
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)



print("混淆矩陣:\n", conf_matrix)
print("準確性:", accuracy)


混淆矩陣:
 [[94 11]
 [26 48]]
準確性: 0.7932960893854749


In [21]:
# 移除不需要的欄位 (e.g., 'index' 和 Column 1) 確保只保留 'PassengerId' 和 'Survived'
output = output[['PassengerId', 'Survived']]

# 將結果儲存為 CSV 檔案，並確認沒有索引欄位
output.to_csv('submission.csv', index=False)
output


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,1
